In [1]:
import pandas as pd
import string
import re
import numpy as np

from gensim.models import Word2Vec
import networkx as nx
from node2vec import Node2Vec

from collections import defaultdict

import math
import operator

In [3]:
df_rec=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/MealRec/recipe.csv")

In [4]:
df_rec.head()

,Unnamed: 0,recipe_id,recipe_name,review_nums,category,aver_rate,image_url,ingredients,cooking_directions,nutritions,reviews,tags
0,0,6698,Mom's Zucchini Bread,9355,appetizer,4.754207,https://images.media-allrecipes.com/userphotos...,all-purpose flour^salt^baking soda^baking powd...,{'directions': u'Prep\n20 m\nCook\n1 h\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{9568256: {'rating': 5, 'followersCount': 0, '...",north-american;breads;easy;beginner-cook;inexp...
1,1,6709,Zucchini Walnut Bread,58,appetizer,4.478873,http://images.media-allrecipes.com/userphotos/...,chopped walnuts^eggs^white sugar^vegetable oil...,"{'directions': u'Whisk together flour, baking ...","{u'niacin': {u'hasCompleteData': True, u'name'...","{1636096: {'rating': 4, 'followersCount': 0, '...",weeknight;breads;fruit;vegetables;kid-friendly...
2,2,6725,Honey Wheat Bread I,2054,appetizer,4.766846,http://images.media-allrecipes.com/userphotos/...,rapid rise yeast^white sugar^warm water (110 d...,{'directions': u'Prep\n25 m\nCook\n35 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1542146: {'rating': 4, 'followersCount': 56, ...",healthy;breads;dietary;low-cholesterol;healthy...
3,3,6730,Chocolate Chip Orange Zucchini Bread,503,dessert,4.674833,https://images.media-allrecipes.com/userphotos...,eggs^white sugar^vegetable oil^vanilla extract...,"{'directions': u'Sift together flour, baking p...","{u'niacin': {u'hasCompleteData': True, u'name'...","{1978376: {'rating': 5, 'followersCount': 0, '...",for-large-groups;healthy;breads;kid-friendly;d...
4,4,6732,Chocolate Chip Pumpkin Bread,866,dessert,4.718112,https://images.media-allrecipes.com/userphotos...,white sugar^pumpkin puree^vegetable oil^water^...,{'directions': u'Prep\n30 m\nCook\n1 h\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1830917: {'rating': 3, 'followersCount': 0, '...",breads;fall;winter;seasonal;quick-breads;4-hou...


In [5]:
df_meal=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\MealRec\meal.csv")

In [6]:
df_meal.head()

,bundle_id,appetizer,main_dish,dessert
0,242,8626,8652,17652
1,885,15034,25272,24843
2,1077,24745,26460,17652
3,1187,15184,55872,19017
4,1340,13062,13218,10813


In [7]:
appetizers=df_meal['appetizer'].values
main=df_meal['main_dish'].values
desert=df_meal['dessert'].values

In [8]:
ids=df_rec['recipe_id'].values
titles=df_rec['recipe_name'].values
id2title={ids[i]:titles[i] for i in range(len(ids))}
title2id={titles[i]:ids[i] for i in range(len(ids))}
id2id={ids[i]:i+1 for i in range(len(ids))}

In [9]:
sentences=[]
for i in range(len(df_meal)):
    sentences.append([
        id2id[appetizers[i]],
        id2id[main[i]],
        id2id[desert[i]],
    ])

In [2]:
df_recs=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Processed\dishes.csv")

In [3]:
df_recs.head()

,ID,Title,Category,Cuisine,Calories,Fats,Proteins,Carbohydrates,Breakfast,Lunch,Snacks,Dinner,Typical_serving_size
0,1,Mom'S Zucchini Bread,appetizer,North America,255.1692,13.140870,3.268513,32.05494,1,0,0,0,100
1,2,Zucchini Walnut Bread,appetizer,NaN,276.0908,13.398670,3.977222,36.54568,1,0,0,0,100
2,3,Honey Wheat Bread I,appetizer,NaN,156.4551,4.770610,4.626510,24.45452,0,1,0,1,100
3,4,Chocolate Chip Orange Zucchini Bread,dessert,NaN,280.3475,15.283680,3.563895,34.25740,1,0,0,0,100
4,5,Chocolate Chip Pumpkin Bread,dessert,NaN,210.1581,9.410045,2.556006,30.47000,1,0,0,0,100


In [4]:
df_recs.describe()

,ID,Calories,Fats,Proteins,Carbohydrates,Breakfast,Lunch,Snacks,Dinner,Typical_serving_size
count,7280.000000,7280.000000,7280.000000,7280.000000,7280.000000,7280.000000,7280.000000,7280.000000,7280.000000,7280.0
mean,3640.500000,333.642836,17.808948,14.245719,29.768142,-0.118544,0.487500,-0.031044,0.660852,100.0
std,2101.699312,207.272847,14.088592,13.908022,23.565712,0.396560,0.734679,0.507802,0.716961,0.0
min,1.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,100.0
25%,1820.750000,179.506925,7.479938,3.424565,12.230360,0.000000,0.000000,0.000000,1.000000,100.0
50%,3640.500000,303.157600,14.562380,8.816630,25.416140,0.000000,1.000000,0.000000,1.000000,100.0
75%,5460.250000,449.430725,24.784125,22.941025,41.857518,0.000000,1.000000,0.000000,1.000000,100.0
max,7280.000000,2825.992000,133.601500,146.447800,568.341700,1.000000,1.000000,1.000000,1.000000,100.0


In [12]:
df_kgl=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Kaggle Meal Plan\mealplans.csv")

In [13]:
df_kgl.head()

,id,calories,caloriesFromFat,totalFat,saturatedFat,cholesterol,sodium,potassium,totalCarbohydrates,dietaryFiber,...,lunch4,dinner0,dinner1,dinner2,dinner3,dinner4,dinner5,dinner6,snacks0,snacks1
0,150001,2455.0,1156.0,128.5,50.4,694.0,4477.0,3377.0,239.8,17.6,...,NaN,24597,31527.0,36455.0,NaN,NaN,NaN,NaN,22025.0,45908.0
1,150002,3515.0,1291.0,143.5,49.1,524.0,7766.0,3734.0,408.1,37.2,...,NaN,21650,24030.0,37700.0,41204.0,NaN,NaN,NaN,17404.0,41265.0
2,150003,3518.0,1709.0,190.0,63.3,1079.0,5261.0,2670.0,354.1,21.6,...,NaN,6145,12578.0,24211.0,48024.0,NaN,NaN,NaN,16607.0,21810.0
3,150004,3771.0,1207.0,134.0,44.3,294.0,3515.0,4667.0,560.8,44.3,...,NaN,16377,20131.0,33967.0,40897.0,NaN,NaN,NaN,35313.0,46133.0
4,150005,3439.0,1191.0,132.2,32.7,751.0,3617.0,6585.0,399.0,35.3,...,NaN,13505,13760.0,31568.0,49135.0,NaN,NaN,NaN,39083.0,NaN


In [14]:
df_kgl_dishes=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Kaggle Meal Plan\dishes.csv")

In [15]:
df_kgl_dishes.head()

,id,title,rating
0,1,1-2-3 Cherry Poke Cake,2.293924
1,2,1-2-3 Complete Breakfast Smoothie,4.848926
2,3,1-2-3 Jambalaya,2.098743
3,4,1-2-3 Lemon Icebox Pie,4.848926
4,5,1-Dish Chicken Parmesan,0.672865


In [16]:
remove_punc=['!','"','.','?',':','/','*','~','#','$','%',';','+','=','[',']','<','>','@','`','^','\\','_','{','}','|']
remove_punc_str="".join(remove_punc) 

def clean(s):
  s = s.title() # convert to titlecase: All individual words first letter capitalised
  s=s.strip() # Remove trailing and leading spaces
  s=s.lstrip(string.digits+" "+string.punctuation) # remove leading numbers, spaces and punctuations
  s = s.translate(str.maketrans({key: None for key in remove_punc_str})) # remove other unnecessary punctuations
  if(len(s)>3):
    if(s[:2]=="A " or s[:3]=="A'S" or s[:2]=="A-" or s[:2]=="A)" or s[:2]=="A1" or s[:3]=="(A)"):
      s=s[2:]
  else:
    return ''
  s=s.lstrip(string.digits+" "+string.punctuation) # remove leading numbers, spaces and punctuations
  s=s.rstrip(" "+remove_punc_str)  # remove trailing spaces and punctuations. Not digits as titles can end with <recipe> for 1
  s=re.sub(' +', ' ', s) # remove extra spaces from in between
  return s

In [17]:
kgl_titles=df_kgl_dishes['title'].values
kgl_ids=df_kgl_dishes['id'].values
kgl_titles=[clean(val) for val in kgl_titles]
kgl_id2title={kgl_ids[i]:kgl_titles[i] for i in range(len(kgl_titles))}

In [18]:
df_kgl.columns

Index(['id', 'calories', 'caloriesFromFat', 'totalFat', 'saturatedFat',
       'cholesterol', 'sodium', 'potassium', 'totalCarbohydrates',
       'dietaryFiber', 'protein', 'sugars', 'vitaminA', 'vitaminC', 'calcium',
       'iron', 'thiamin', 'niacin', 'vitaminB6', 'magnesium', 'folate',
       'breakfast0', 'breakfast1', 'breakfast2', 'lunch0', 'lunch1', 'lunch2',
       'lunch3', 'lunch4', 'dinner0', 'dinner1', 'dinner2', 'dinner3',
       'dinner4', 'dinner5', 'dinner6', 'snacks0', 'snacks1'],
      dtype='object')

In [19]:
meal_plans=df_kgl[['breakfast0', 'breakfast1', 'breakfast2', 'lunch0', 'lunch1', 'lunch2',
       'lunch3', 'lunch4', 'dinner0', 'dinner1', 'dinner2', 'dinner3',
       'dinner4', 'dinner5', 'dinner6', 'snacks0', 'snacks1']].values.tolist()

In [20]:
len(meal_plans),len(df_kgl)

(275290, 275290)

In [21]:
titles=df_recs['Title'].values
titles=[clean(val) for val in titles]
title2id={titles[i]:i+1 for i in range(len(titles))}
titles=set(titles)

In [22]:
print(meal_plans[0])

[16871.0, 51793.0, nan, 40536.0, 44745.0, nan, nan, nan, 24597.0, 31527.0, 36455.0, nan, nan, nan, nan, 22025.0, 45908.0]


In [23]:
sentences=[]
for i in range(len(df_meal)):
    sentences.append([
        id2id[appetizers[i]],
        id2id[main[i]],
        id2id[desert[i]],
    ])

In [24]:
for meal_plan in meal_plans:
    flag=1
    sent=[]
    for dish in meal_plan:
        if(not np.isnan(float(dish))):
            if kgl_id2title[int(dish)] in titles:
                sent.append(title2id[kgl_id2title[int(dish)]])
    if len(sent)>1:
        sentences.append(sent)

In [25]:
len(sentences)

110301

In [66]:
model_w2v = Word2Vec(sentences, min_count=1)

In [26]:
model_w2v.wv.save_word2vec_format("F:\SHRINIVAS\KGP\BTP\Meal-Planning\models\w2v_combi.bin",binary=True)

In [26]:
print(model_w2v)

Word2Vec<vocab=6350, vector_size=100, alpha=0.025>


In [101]:
titles=df_recs['Title'].values
titles=[clean(val) for val in titles]
id2title={i+1:titles[i] for i in range(len(titles))}

In [99]:
def getSimilarW2v(id):
    print("Title: ",titles[id-1])
    sims=model_w2v.wv.similar_by_word(id)
    for id in sims:
        print(id2title[id[0]])

In [100]:
getSimilarW2v(10)

Title:  Italian Herb Bread I
Pepperoni Stuffed Mushrooms
Parmesan Puffs
Italian Tomato Cucumber Salad
Root Vegetables Baked In Pesto Sauce
Cranberry Almond Biscotti
Fantastic Focaccia Bread
Fresh Mozzarella Pasta Salad
Party Italian Wedding Soup
Spoon Cookies
Easy Polenta With Tomato Sauce


In [ ]:
df_recs['Cuisine'].unique()

array(['North America', nan, 'United States', 'Europe', 'Italy',
       'Middle East', 'South East Asia', 'Canada', 'France', 'Mexico',
       'British Isles', 'Australia & NZ', 'Greece', 'Eastern Europe',
       'Asia', 'South America', 'China', 'Japan', 'Africa',
       'Indian Subcontinent', 'Korea'], dtype=object)

In [ ]:
df_recs[df_recs['Cuisine']=='Indian Subcontinent'].sample(5)

,ID,Title,Category,Cuisine,Calories,Fats,Proteins,Carbohydrates,Breakfast,Lunch,Snacks,Dinner,Typical_serving_size
2677,2678,Stuffed Tomatoes,appetizer,Indian Subcontinent,548.0551,32.773380,28.487350,36.73273,0,1,0,1,100
5892,5893,Aloo Gobhi,main-dish,Indian Subcontinent,244.1541,7.679012,8.205558,40.48901,0,1,0,1,100
5319,5320,Tim Perry'S Soup (Creamy Curry Cauliflower And...,appetizer,Indian Subcontinent,348.4337,20.019760,20.725330,24.08469,0,1,0,1,100
4342,4343,Chicken Korma Ii,main-dish,Indian Subcontinent,397.7891,27.461760,25.311290,13.44186,0,1,0,1,100
3890,3891,Chicken Makhani (Indian Butter Chicken),main-dish,Indian Subcontinent,407.8225,27.773570,23.377060,15.61461,0,1,0,1,100


In [ ]:
getSimilarW2v(2497)

Title:  Kulfi
Napa Cabbage Noodle Salad
Fudgy Chocolate Chip Muffins
Banana Pudding Iv
Maple Cookies
Best Big, Fat, Chewy Chocolate Chip Cookie
Low 'N Slow Mushroom Barley Soup
Indonesian Fried Rice (Nasi Goreng)
Cod Au Gratin
Southwestern Chicken Pitas With Chipotle Sauce
Tofu Salad


### Graph Based Approach

In [ ]:
len(sentences)

110301

In [130]:
sentences[0],sentences[-1]

([316, 326, 2212], [6721, 6226, 1496])

In [45]:
def normalize(d):
    factor=max(d.items(),key=lambda x:x[1])[1]
    for k in d:
        d[k]/=factor   

In [51]:
G=nx.Graph()

In [52]:
nodes=[i+1 for i in range(len(ids))]
G.add_nodes_from(nodes)

In [53]:
scores=defaultdict(float)
for sent in sentences:
    for i in range(len(sent)):
        for j in range(len(sent)):
            if(i!=j):
                scores[(sent[i],sent[j])]+=1

print(len(scores))

normalize(scores)
avg=math.fsum(scores.values())/len(scores)
print(avg)

edges=[]
for sc in scores.items():
    if sc[1]>avg*2:
        edges.append((sc[0][0],sc[0][1],sc[1]))

print(len(edges))

400797
0.02655787888087531
13296


In [54]:
G.add_weighted_edges_from(edges)

In [55]:
len(G.edges)

6700

In [56]:
node2vec = Node2Vec(G, dimensions=64, walk_length=5, num_walks=20, workers=4,p=1,q=0.25)

Computing transition probabilities:   0%|          | 0/7280 [00:00<?, ?it/s]

In [57]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [87]:
model.wv.save_word2vec_format("F:\SHRINIVAS\KGP\BTP\Meal-Planning\models\graph_combi.bin",binary=True)

In [58]:
scores=defaultdict(float)
for sent in sentences:
    for i in range(len(sent)):
        for j in range(len(sent)):
            if(i!=j):
                scores[(sent[i],sent[j])]+=1

In [59]:
def find(id):
    scores=defaultdict(int)
    for sent in sentences:
        if id in sent:
            for i in sent:
                if i!=id:
                    scores[i]+=1
    return scores

In [72]:
def getSimilar(id):
    print("Title: ",titles[id-1])
    sims=model.wv.similar_by_word(str(id))
    for i in sims:
        print(titles[int(i[0])-1],scores[(int(i[0]),id)],scores[(id,int(i[0]))],i[1])

In [82]:
def getSimilarW2v(id):
    print("Title: ",titles[id-1])
    sims=model_w2v.wv.similar_by_word(id)
    for i in sims:
        print(id2title[i[0]],scores[(int(i[0]),id)],scores[(id,int(i[0]))],i[1])

In [73]:
getSimilar(10)

Title:  Italian Herb Bread I
Lemon Poppy Seed Biscotti 3.0 3.0 0.9299562573432922
Marinated Olives 3.0 3.0 0.9257369041442871
Buffalo Chicken Fingers 0.0 0.0 0.9154437184333801
Antipasto Pasta Salad 0.0 0.0 0.9149659872055054
Bocconcini Salad 0.0 0.0 0.9111578464508057
Garlic-Ginger Chicken Wings 0.0 0.0 0.90390545129776
Tomozzakala Salad 1.0 1.0 0.9037086963653564
American Lasagna 0.0 0.0 0.9032275080680847
Pizza Balls 0.0 0.0 0.9029725193977356
Bolognese Sauce 0.0 0.0 0.9022630453109741


In [83]:
getSimilar(2497)

Title:  Kulfi
Chocolate Orange Truffles 0.0 0.0 0.4825442433357239
Creamy Pumpkin Sauce 0.0 0.0 0.46662598848342896
Stuffed Mexican Peppers 0.0 0.0 0.44891971349716187
Chocolate Chip Kiss Cookies 0.0 0.0 0.4318309724330902
White Wine And Mushroom Sauce 0.0 0.0 0.42909401655197144
Caramel Apple Bars Ii 0.0 0.0 0.39895278215408325
Artichoke Asparagus Pasta Salad 0.0 0.0 0.3724807798862457
Buffalo Wontons 0.0 0.0 0.3668472468852997
Italian Sausage Penne 0.0 0.0 0.3650813400745392
Day-After-Thanksgiving Turkey Carcass Soup 0.0 0.0 0.36248132586479187


### Comparison

In [13]:
from gensim.models.keyedvectors import KeyedVectors

In [14]:
graph_combi=KeyedVectors.load_word2vec_format("F:\SHRINIVAS\KGP\BTP\Meal-Planning\models\graph_combi.bin",binary=True)
w2v_combi=KeyedVectors.load_word2vec_format("F:\SHRINIVAS\KGP\BTP\Meal-Planning\models\w2v_combi.bin",binary=True)

In [15]:
graph_combi.similarity(10,20)

0.4307086

In [138]:
def getSimilar(id):
    print("Title: ",titles[id-1])
    sims=graph_combi.similar_by_word(str(id))
    for i in sims:
        print(titles[int(i[0])-1])##,scores[(int(i[0]),id)],scores[(id,int(i[0]))],i[1])

In [137]:
def getSimilarW2v(id):
    print("Title: ",titles[id-1])
    sims=w2v_combi.similar_by_word(id)
    for i in sims:
        print(titles[int(i[0])-1])##,scores[(int(i[0]),id)],scores[(id,int(i[0]))],i[1])

In [139]:
getSimilar(5423)

Title:  Jar Doo Chicken Wings
Sticky Sesame Chicken
Best Big, Fat, Chewy Chocolate Chip Cookie
Curry Kabobs
Blonde Brownies I
Cabbage Rolls Ii
Pan-Seared T-Bone For Two With Rosemary Mustard Sauce
Creamy Potato Leek Soup Ii
Pressure Cooker Beef Stew
Low 'N Slow Mushroom Barley Soup
Mushroom, Leek, Chicken Sausage And Tortellini Soup


In [140]:
getSimilarW2v(5423)

Title:  Jar Doo Chicken Wings
Honey Mustard Grilled Chicken
Greek Penne And Chicken
Pumpkin Cream Cheese Muffins
Cod Curry
Deviled Eggs I
Creamed Eggs On Toast
Indonesian Spiced Rice
Andouille And Chicken Creole Pasta
Easiest Peanut Butter Fudge
Peppy'S Pita Bread


In [141]:
topK=sorted(find(5423).items(),key=lambda x:x[1],reverse=True)[:10]
for k in topK:
    print(titles[k[0]-1],k[1])##,k[0])

Best Big, Fat, Chewy Chocolate Chip Cookie 3
Mushroom, Leek, Chicken Sausage And Tortellini Soup 2
Delicious Ham And Potato Soup 2
Christmas Pinwheel Cookies 2
Caribbean-Spiced Roast Chicken 2
Pumpkin And Butternut Creamed Soup 2
Grilled Marinated Swordfish 2
Lemon Turkey Soup 2
French Onion Soup Xi 2
Blueberry Crumb Bars 2


In [2]:
import pandas as pd

In [3]:
df_dishes=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/dishes.csv")

In [5]:
dishes=df_dishes.to_dict('records')

In [11]:
type(dishes[0]['Calories'])

float

In [7]:
id2dish={}
for dish in dishes:
    id2dish[dish['ID']]={
        'Title': dish['Title'],
        'Nutrition': [dish['Calories'],dish['Fats'],dish['Proteins'],dish['Carbohydrates']],
        'Serving Size': dish['Typical_serving_size']
    }

In [8]:
len(id2dish)

7280

In [12]:
'Hel.lo'.replace('.',' ')

'Hel lo'